In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn import preprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from tensorflow.keras.layers import Dense, Conv2D , SeparableConv2D, MaxPool2D, Flatten , Dropout , BatchNormalization
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow import keras

from scipy.stats import zscore
from bayes_opt import BayesianOptimization
import keras_tuner as kt



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-delays-prediction-challeng2021/airports.csv
/kaggle/input/flight-delays-prediction-challeng2021/airlines.csv
/kaggle/input/flight-delays-prediction-challeng2021/flights_test.csv
/kaggle/input/flight-delays-prediction-challeng2021/flights_train.csv
/kaggle/input/flight-delays-prediction-challeng2021/submit_sample.csv
/kaggle/input/aml-model/first_model/__notebook_source__.ipynb
/kaggle/input/aml-model/first_model/new_model/saved_model.pb
/kaggle/input/aml-model/first_model/new_model/keras_metadata.pb
/kaggle/input/aml-model/first_model/new_model/variables/variables.index
/kaggle/input/aml-model/first_model/new_model/variables/variables.data-00000-of-00001
/kaggle/input/aml-model/new_model/new_model/saved_model.pb
/kaggle/input/aml-model/new_model/new_model/keras_metadata.pb
/kaggle/input/aml-model/new_model/new_model/variables/variables.index
/kaggle/input/aml-model/new_model/new_model/variables/variables.data-00000-of-00001
/kaggle/input/aml-model/garbage/saved_mod

In [2]:
train = pd.read_csv('../input/aml-project/flights_training.csv')
test = pd.read_csv('../input/aml-project/flights_testing.csv')
val = pd.read_csv('../input/aml-project/flights_validation.csv')

In [3]:
airp = pd.read_csv('../input/flight-delays-prediction-challeng2021/airports.csv')

In [4]:
airp = airp.set_index('IATA_CODE')
airp.at['ECP','LATITUDE'] = 30.176592
airp.at['ECP','LONGITUDE'] = -85.805489
airp.at['PBG','LATITUDE'] = 44.659709
airp.at['PBG','LONGITUDE'] = -73.467220
airp.at['UST','LATITUDE'] = 29.958600
airp.at['UST','LONGITUDE'] = -81.338900
airp = airp.reset_index()


In [5]:
airp.to_csv('airports.csv')

In [6]:
airp[airp['LATITUDE'].isnull()]

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE


def split_data():
    
    #CHANGE INPUT LOCATION
    airports = airp_and_airp
    flights_train = pd.read_csv('../input/flight-delays-prediction-challeng2021/flights_train.csv')
    
    airports_origin = airports[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'ORIGIN_AIRPORT'})
    airports_arrive = airports[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'DESTINATION_AIRPORT'})
    flights_train = flights_train.merge(airports_origin, on = 'ORIGIN_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_origin', 'LONGITUDE' : 'LONGITUDE_origin'})
    flights_train = flights_train.merge(airports_arrive, on = 'DESTINATION_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_arrival', 'LONGITUDE' : 'LONGITUDE_arrival'})
    
    train, test_val = train_test_split(flights_train, test_size=0.20, random_state = 42)
    test, val = train_test_split(test_val, test_size=0.6, random_state = 42)
    print('splitted data into parts with sizes: {} train, {} test and {} validation'.format(len(train), len(test), len(val)))
    
    #CHANGE OUTPUT NAMES
    train.to_csv('flights_training.csv')
    test.to_csv('flights_testing.csv')
    val.to_csv('flights_validation.csv')

name_list = ['ATL',
 'BNA',
 'BWI',
 'DAL',
 'ECP',
 'FLL',
 'HOU',
 'IAD',
 'IAH',
 'MYR',
 'PBG',
 'STL',
 'TTN',
 'UST']

new_df = {}
new_df['AIRPORTS'] = []
new_df['LONGITUDE'] = []
new_df['LATITUDE'] = []
for i in range(len(lat_long_list)):
    airport = name_list[i]
    lat, long = lat_long_list[i]
    new_df['AIRPORTS'].append(airport)
    new_df['LONGITUDE'].append(long)
    new_df['LATITUDE'].append(lat)

lat_long_list = [
[33.640411, -84.419853],
[36.131687, -86.668823],
[39.177404, -76.668392],
[32.848152,-96.851349],
[30.176592, -85.805489],
[26.139412, -80.133591],
[29.652805, -95.267243],
[38.944555, -77.455266],
[29.980498, -95.339657],
[33.682202, -78.927894],
[44.659709, -73.467220],
[38.751826, -90.3708],
[40.277031, -74.818049],
[29.9586, -81.3389]]
    

In [7]:
#!tar -czvf ./new_model.tar.gz ./

def build_model(list_of_params, input_shape):
    model = Sequential()
    n_param = len(list_of_param)
    for i, param in enumerate(list_of_params):
        if i == 0:
            model.add(Dense(param, input_shape=input_shape, kernel_initializer='normal', activation='relu'))
        elif i == n_param - 1:
            model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
        else:
            model.add(Dense(param, kernel_initializer='normal', activation = 'relu'))
            
    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())    
            
    return model

from scipy.stats import zscore
from bayes_opt import BayesianOptimization
import keras_tuner as kt

train = pd.read_csv('../input/aml-project/AML_data/flights_training.csv')
test = pd.read_csv('../input/aml-project/AML_data/flights_testing.csv')
val = pd.read_csv('../input/aml-project/AML_data/flights_validation.csv')

In [8]:
remove_outliers = False

train = train.dropna()
val = val.dropna()
test= test.dropna()

cols = train.columns
cont_cols = list(cols[11:18]) + list(cols[19:])
cat_cols = list(cols[4:7])
print(cont_cols, cat_cols)

s_scaler2 = preprocessing.StandardScaler()
random_list = s_scaler2.fit_transform(train['ARRIVAL_DELAY'].values.reshape((-1,1)))

cont_cols_del = cont_cols + ['ARRIVAL_DELAY']

s_scaler = preprocessing.StandardScaler()
train[cont_cols_del] = s_scaler.fit_transform(train[cont_cols_del])
val[cont_cols_del] = s_scaler.fit_transform(val[cont_cols_del])
test[cont_cols] = s_scaler.fit_transform(test[cont_cols])
train = pd.get_dummies(train, columns = cat_cols)
val = pd.get_dummies(val, columns = cat_cols)
test = pd.get_dummies(test, columns = cat_cols)

new_vars = list(train.columns[8:15]) + list(train.columns[19:])
print(new_vars)
input_length = len(new_vars)

if remove_outliers:
            z_scores = zscore(train['ARRIVAL_DELAY'])
            abs_z_scores = np.abs(z_scores)
            filtered_entries = (abs_z_scores < 3)
            train = train[filtered_entries]
            
input_shape = (input_length, )

['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'LATITUDE_origin', 'LONGITUDE_origin', 'LATITUDE_arrival', 'LONGITUDE_arrival'] ['DAY', 'DAY_OF_WEEK', 'AIRLINE']
['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'LONGITUDE_arrival', 'DAY_1', 'DAY_2', 'DAY_3', 'DAY_4', 'DAY_5', 'DAY_6', 'DAY_7', 'DAY_8', 'DAY_9', 'DAY_10', 'DAY_11', 'DAY_12', 'DAY_13', 'DAY_14', 'DAY_15', 'DAY_16', 'DAY_17', 'DAY_18', 'DAY_19', 'DAY_20', 'DAY_21', 'DAY_22', 'DAY_23', 'DAY_24', 'DAY_25', 'DAY_26', 'DAY_27', 'DAY_28', 'DAY_29', 'DAY_30', 'DAY_31', 'DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5', 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'AIRLINE_AA', 'AIRLINE_AS', 'AIRLINE_B6', 'AIRLINE_DL', 'AIRLINE_EV', 'AIRLINE_F9', 'AIRLINE_HA', 'AIRLINE_MQ', 'AIRLINE_NK', 'AIRLINE_OO', 'AIRLINE_UA', 'AIRLINE_US', 'AIRLINE_VX', 'AIRLINE_WN']


In [9]:
model = Sequential()
model.add(Dense(600, input_shape=input_shape, kernel_initializer='normal', activation='relu'))
model.add(Dense(450, kernel_initializer='normal', activation='relu'))
model.add(Dense(300, kernel_initializer='normal', activation='relu'))
model.add(Dense(200, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))
# Compile model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [10]:
submit_sample = pd.read_csv('../input/flight-delays-prediction-challeng2021/flights_test.csv')
submit_sample.head()

,id,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL
0,0,2015,7,1,3,NK,298,N624NK,LAS,IAH,3,41.0,11.0,52.0,177.0,1222,500
1,1,2015,7,1,3,AA,1965,N507AY,SFO,CLT,5,2400.0,21.0,21.0,303.0,2296,808
2,2,2015,7,1,3,DL,2020,N3759,LAX,MEM,5,2400.0,36.0,36.0,223.0,1619,548
3,3,2015,7,1,3,NK,612,N629NK,LAS,MSP,5,53.0,14.0,107.0,178.0,1299,503
4,4,2015,7,1,3,UA,1044,N39418,ANC,ORD,8,17.0,20.0,37.0,364.0,2846,912


In [11]:
submit_sample = pd.read_csv('../input/flight-delays-prediction-challeng2021/flights_test.csv')
print(len(submit_sample))
airports_origin = airp[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'ORIGIN_AIRPORT'})
airports_arrive = airp[['IATA_CODE','LATITUDE','LONGITUDE']].rename(columns = {'IATA_CODE' : 'DESTINATION_AIRPORT'})
submit_sample1 = submit_sample.merge(airports_origin, on = 'ORIGIN_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_origin', 'LONGITUDE' : 'LONGITUDE_origin'})
submit_sample2 = submit_sample1.merge(airports_arrive, on = 'DESTINATION_AIRPORT').rename(columns = {'LATITUDE' : 'LATITUDE_arrival', 'LONGITUDE' : 'LONGITUDE_arrival'})
print(len(submit_sample2))
s_scaler = preprocessing.StandardScaler()
submit_sample2[cont_cols] = s_scaler.fit_transform(submit_sample2[cont_cols])
submit_sample2 = pd.get_dummies(submit_sample2, columns = cat_cols)
submit_sample.head()

514384
514384


,id,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL
0,0,2015,7,1,3,NK,298,N624NK,LAS,IAH,3,41.0,11.0,52.0,177.0,1222,500
1,1,2015,7,1,3,AA,1965,N507AY,SFO,CLT,5,2400.0,21.0,21.0,303.0,2296,808
2,2,2015,7,1,3,DL,2020,N3759,LAX,MEM,5,2400.0,36.0,36.0,223.0,1619,548
3,3,2015,7,1,3,NK,612,N629NK,LAS,MSP,5,53.0,14.0,107.0,178.0,1299,503
4,4,2015,7,1,3,UA,1044,N39418,ANC,ORD,8,17.0,20.0,37.0,364.0,2846,912


In [12]:
model = keras.models.load_model('../input/aml-model/garbage')

In [13]:
submit_sample2['AIRLINE_US'] = 0

In [14]:
test_preds = model.predict(submit_sample2[new_vars])
#test_true = submit_sample1['ARRIVAL_DELAY']
test_inverse = s_scaler2.inverse_transform(test_preds)
#r2_score(test_true, test_inverse)

2021-12-10 10:38:41.105146: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


test_preds = model.predict(test[new_vars])
test_true = test['ARRIVAL_DELAY']
test_inverse = s_scaler2.inverse_transform(test_preds)
r2_score(test_true, test_inverse)

test_true[:5]

In [15]:
test_inverse[:5]

array([[ 18.891657],
       [ 28.896254],
       [-14.063471],
       [-12.282604],
       [ -9.72913 ]], dtype=float32)

from sklearn.metrics import mean_squared_error

rms = mean_squared_error(test_true, test_inverse, squared=True)
print(rms)

In [16]:
sss = pd.read_csv('../input/flight-delays-prediction-challeng2021/submit_sample.csv')
sss.head()

,id,ARRIVAL_DELAY
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [17]:
id_list = np.arange(len(test_inverse))
print(len(id_list))

514384


In [18]:
test_inverse.flatten().shape

(514384,)

In [19]:
ss = pd.DataFrame({'id' : id_list,
                  'ARRIVAL_DELAY' : test_inverse.flatten()})
ss = ss.set_index('id')

In [20]:
ss.to_csv('submission.csv')

def model_builder(hp, input_shape = input_shape):
    model = keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=input_shape))
    
    for i in range(6):
        model.add(tf.keras.layers.Dense(units=hp.Choice('units_' + str(i), 
                                                        values=[600,400,300,200,100,50,25][i:i+2]),
                                        activation='relu'))

    model.add(Dense(1, kernel_initializer='normal', activation = 'linear'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adadelta(learning_rate=hp_learning_rate),
            loss='mean_squared_error',
            metrics=[
                    'MeanSquaredError'
                    ],
            steps_per_execution=8
                 )
    
    return model

tuner = kt.Hyperband(model_builder,
                     objective='val_mean_squared_error',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner.search(train[new_vars], 
             train['ARRIVAL_DELAY'], 
             epochs=60, 
             validation_data=(val[new_vars], val['ARRIVAL_DELAY']),
             callbacks = [stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

def optimize_model(train_data, val_data, input_size = input_shape, input_columns = new_vars):
    
    def dense_training(train_df, val_df, list_of_params, input_size, input_columns, 
                       epochs, remove_outliers, batch_size, target_column = 'ARRIVAL_DELAY'):
        
        if remove_outliers:
            z_scores = zscore(train_df[target_column])
            abs_z_scores = np.abs(z_scores)
            filtered_entries = (abs_z_scores < 3)
            train_df = train_df[filtered_entries]
        
        stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        
        model = build_model(list_of_params, input_size)
        result = model.fit(train_df[input_columns], train_df[target_column], 
                        epochs=epochs, batch_size=batch_size, 
                        callbacks=[stop_early],
                        validation_split=(val_df[input_columns],val_df[target_column]))
        return result.history['val_loss']
    
                           
    optimizer = BayesianOptimization(
    f=dense_training,
    pbounds={
        'train_df': train_data,
        'val_df' : val_data,
        'input_size' : input_size,
        'input_columns' : input_columns,
        'remove outliers' : [True, False],
        'list_of_params': [[200,100,50,25],[400,200,100,50,25],[800,400,200,100,50,25],[600,450,300,150,75,25]], 
        'epochs': (10,20,50,80,100),
        'batch_size': (16,32,64)
            },
    random_state = 12,
    verbose=2
    )
                           
    optimizer.maximize(n_iter=10)

    print("Final result:", optimizer.max)

tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=100,
                     factor=3,
                     hyperband_iterations=10,
                     directory="kt_dir",
                     project_name="kt_hyperband",)